# GR00T Inference

This tutorial shows how to use the GR00T inference model to predict the actions from the observations, given a test dataset.

In [ ]:
!git clone https://github.com/NVIDIA/Isaac-GR00T

Cloning into 'Isaac-GR00T'...
remote: Enumerating objects: 445, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 445 (delta 159), reused 97 (delta 97), pack-reused 206 (from 2)
Receiving objects: 100% (445/445), 48.19 MiB | 39.96 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [ ]:
!uv venv && source .venv/bin/activate && cd Isaac-GR00T && uv pip install --upgrade setuptools && uv pip install -e .[base] && uv pip install --no-build-isolation flash-attn==2.7.1.post4

Streaming output truncated to the last 5000 lines.
nvidia-cusolver-cu12 ------------------------------ 22.98 MiB/122.01 MiB
nvidia-cusparse-cu12 ------------------------------ 22.90 MiB/197.84 MiB
triton     ------------------------------ 23.11 MiB/199.80 MiB
nvidia-cufft-cu12 ------------------------------ 23.07 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 23.07 MiB/346.60 MiB
tensorflow ------------------------------ 23.09 MiB/453.24 MiB
nvidia-cudnn-cu12 ------------------------------ 23.04 MiB/633.96 MiB
⠸ Preparing packages... (45/61)
onnx       ------------------------------ 3.88 MiB/14.99 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 23.44 MiB/23.50 MiB
av         ------------------------------ 23.23 MiB/32.82 MiB
pyarrow    ------------------------------ 23.47 MiB/36.23 MiB
nvidia-curand-cu12 ------------------------------ 23.57 MiB/53.70 MiB
opencv-python ------------------------------ 23.12 MiB/58.85 MiB
ray        -----------------------------

In [ ]:
import os
import torch
import gr00t

from gr00t.data.dataset import LeRobotSingleDataset
from gr00t.model.policy import Gr00tPolicy

ModuleNotFoundError: No module named 'gr00t'

In [ ]:
# change the following paths
MODEL_PATH = "nvidia/GR00T-N1.5-3B"

# REPO_PATH is the path of the pip install gr00t repo and one level up
REPO_PATH = os.path.dirname(os.path.dirname(gr00t.__file__))
DATASET_PATH = os.path.join(REPO_PATH, "demo_data/robot_sim.PickNPlace")
EMBODIMENT_TAG = "gr1"

device = "cuda" if torch.cuda.is_available() else "cpu"

## Loading Pretrained Policy

Policy Model is loaded just like any other huggingface model.

There are 2 new concepts here in the GR00T model:
 - modality config: This defines the keys in the dictionary used by the model. (e.g. `action`, `state`, `annotation`, `video`)
 - modality_transform: A sequence of transform which are used during dataloading

In [ ]:
from gr00t.experiment.data_config import DATA_CONFIG_MAP


data_config = DATA_CONFIG_MAP["fourier_gr1_arms_only"]
modality_config = data_config.modality_config()
modality_transform = data_config.transform()

policy = Gr00tPolicy(
    model_path=MODEL_PATH,
    embodiment_tag=EMBODIMENT_TAG,
    modality_config=modality_config,
    modality_transform=modality_transform,
    device=device,
)

# print out the policy model architecture
print(policy.model)

## Loading dataset

First this requires user to check which embodiment tags are used to pretrained the `Gr00tPolicy` pretrained models.

In [ ]:
import numpy as np

modality_config = policy.modality_config

print(modality_config.keys())

for key, value in modality_config.items():
    if isinstance(value, np.ndarray):
        print(key, value.shape)
    else:
        print(key, value)


In [ ]:
# Create the dataset
dataset = LeRobotSingleDataset(
    dataset_path=DATASET_PATH,
    modality_configs=modality_config,
    video_backend="decord",
    video_backend_kwargs=None,
    transforms=None,  # We'll handle transforms separately through the policy
    embodiment_tag=EMBODIMENT_TAG,
)

Let's print out a single data and visualize it

In [ ]:
import numpy as np

step_data = dataset[0]

print(step_data)

print("\n\n ====================================")
for key, value in step_data.items():
    if isinstance(value, np.ndarray):
        print(key, value.shape)
    else:
        print(key, value)


Let's plot just the "right hand" state and action data and see how it looks like. Also show the images of the right hand state.

In [ ]:
import matplotlib.pyplot as plt

traj_id = 0
max_steps = 150

state_joints_across_time = []
gt_action_joints_across_time = []
images = []

sample_images = 6

for step_count in range(max_steps):
    data_point = dataset.get_step_data(traj_id, step_count)
    state_joints = data_point["state.right_arm"][0]
    gt_action_joints = data_point["action.right_arm"][0]


    state_joints_across_time.append(state_joints)
    gt_action_joints_across_time.append(gt_action_joints)

    # We can also get the image data
    if step_count % (max_steps // sample_images) == 0:
        image = data_point["video.ego_view"][0]
        images.append(image)

# Size is (max_steps, num_joints == 7)
state_joints_across_time = np.array(state_joints_across_time)
gt_action_joints_across_time = np.array(gt_action_joints_across_time)


# Plot the joint angles across time
fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(8, 2*7))

for i, ax in enumerate(axes):
    ax.plot(state_joints_across_time[:, i], label="state joints")
    ax.plot(gt_action_joints_across_time[:, i], label="gt action joints")
    ax.set_title(f"Joint {i}")
    ax.legend()

plt.tight_layout()
plt.show()


# Plot the images in a row
fig, axes = plt.subplots(nrows=1, ncols=sample_images, figsize=(16, 4))

for i, ax in enumerate(axes):
    ax.imshow(images[i])
    ax.axis("off")


Now we can run the policy from the pretrained checkpoint.

In [ ]:
predicted_action = policy.get_action(step_data)
for key, value in predicted_action.items():
    print(key, value.shape)

### Understanding the Action Output

Each joint in the action output has a shape of (16, N) where N is the degree of freedom for the joint.
- 16 represents the action horizon (predictions for timesteps t, t+1, t+2, ..., t+15)

For each arm (left and right):
- 7 arm joints:
  - Shoulder pitch
  - Shoulder roll
  - Shoulder yaw
  - Elbow pitch
  - Wrist yaw
  - Wrist roll
  - Wrist pitch

For each hand (left and right):
- 6 finger joints:
  - Little finger
  - Ring finger
  - Middle finger
  - Index finger
  - Thumb rotation
  - Thumb bending

For the waist
- 3 joints:
  - torso waist yaw
  - torso waist pitch
  - torso waist roll
